In [3]:
# Mount Google Drive Folder (/content/drive/My Drive/Colab)
# 코랩에서 구글 드라이브를 사용하기 위해 import하는 부분
from google.colab import drive
drive.mount('/content/drive')

# import user defined files 
import sys
import_dir ="/content/drive/My Drive/Colab Notebooks/2. Classification/pytorch-cifar"
sys.path.insert(0, import_dir)

# import ipynb files...
!pip install import_ipynb 
import import_ipynb

%run '/content/drive/My Drive/Colab Notebooks/2. Classification/pytorch-cifar/dataset.ipynb'

import easydict 
args = easydict.EasyDict({ "lr": 0.1, "resume": False})

# Debugging Tool
import pdb

# Download & Unzip Dataset (Move from google drive to local)
# 구글 드라이브에 있는 cifar.tgz 파일을 카피해서 코랩의 로컬(content/sample_data) 안에 저장
%cp -r '/content/drive/My Drive/Colab Notebooks/Dataset/CIFAR10/cifar.tgz' '/content/sample_data'
# 코랩의 로컬에서 압축 해제
!tar -xvf  '/content/sample_data/cifar.tgz'

# Initilize Additional parameters
train_root = "/content/cifar/train"
test_root = "/content/cifar/test"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
cifar/train/49869_airplane.png
cifar/train/43849_airplane.png
cifar/train/42320_airplane.png
cifar/train/35521_truck.png
cifar/train/17901_automobile.png
cifar/train/818_frog.png
cifar/train/17954_bird.png
cifar/train/45941_dog.png
cifar/train/34990_horse.png
cifar/train/6529_ship.png
cifar/train/4851_ship.png
cifar/train/16667_horse.png
cifar/train/29916_horse.png
cifar/train/7214_automobile.png
cifar/train/48391_bird.png
cifar/train/4178_horse.png
cifar/train/9946_automobile.png
cifar/train/37702_automobile.png
cifar/train/29988_bird.png
cifar/train/24061_bird.png
cifar/train/953_truck.png
cifar/train/40243_horse.png
cifar/train/2405_horse.png
cifar/train/26051_bird.png
cifar/train/24741_dog.png
cifar/train/26831_horse.png
cifar/train/16253_airplane.png
cifar/train/25364_ship.png
cifar/train/5395_truck.png
cifar/train/6474_ship.png
cifar/train/36839_horse.png
cifar/train/20818_bird.png
cifar/train/33740_deer.png
cifar/train/46808_truck.png
cifar/tr

In [4]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar

'''
# Removed Code (Because ipynb file does not support argparse)
# Even removed, these lines are substitued by easydict code

parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action='store_true',
                    help='resume from checkpoint')
args = parser.parse_args()
'''

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Modified for custom dataset
#trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainset = Dataset(root=train_root, transforms=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=os.cpu_count(), drop_last=True)

# Modified for custom dataset
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testset = Dataset(root=test_root, transforms=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=os.cpu_count(), drop_last=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
net = MobileNet() # 가장 가벼운 모델
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()


net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')   
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'   
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0    

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() 
        _, predicted = outputs.max(1)        
        total += targets.size(0) 
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        #print(batch_idx,'/', len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #             % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            #print(batch_idx,'/', len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.   
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc



for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)

==> Preparing data..
==> Building model..

Epoch: 0
 [================================================================>]  Step: 45ms | Tot: 28s135ms | Loss: 1.749 | Acc: 35.805% (17874/49920) 390/390 
 [================================================================>]  Step: 24ms | Tot: 4s244ms | Loss: 1.461 | Acc: 47.140% (4714/10000) 100/100 
Saving..

Epoch: 1
 [================================================================>]  Step: 49ms | Tot: 27s642ms | Loss: 1.336 | Acc: 51.635% (25776/49920) 390/390 
 [================================================================>]  Step: 22ms | Tot: 4s103ms | Loss: 1.316 | Acc: 53.100% (5310/10000) 100/100 
Saving..

Epoch: 2
 [================================================================>]  Step: 45ms | Tot: 27s592ms | Loss: 1.102 | Acc: 61.070% (30486/49920) 390/390 
 [================================================================>]  Step: 21ms | Tot: 4s52ms | Loss: 0.959 | Acc: 66.960% (6696/10000) 100/100 
Saving..

Epoch: 3
 [==